**Implementing a Simple GCN**

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCNLayer(nn.Module):
    """
        GCN layer

        Args:
            input_dim (int): Dimension of the input
            output_dim (int): Dimension of the output (a softmax distribution)
            A (torch.Tensor): 2D adjacency matrix
    """

    def __init__(self, input_dim: int, output_dim: int, A: torch.Tensor):
        super(GCNLayer, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.A = A
        self.A_hat = self.A + torch.eye(self.A.size(0))
        self.ones = torch.ones(input_dim, input_dim)
        self.D = torch.matmul(self.A.float(), self.ones.float())
        self.D = torch.diag(self.D)
        self.D = torch.diag_embed(self.D)
        self.D_neg_sqrt = torch.diag_embed(torch.diag(torch.pow(self.D, -0.5)))
        self.W = nn.Parameter(torch.rand(input_dim, output_dim))

    def forward(self, X: torch.Tensor):
        support_1 = torch.matmul(self.D_neg_sqrt, torch.matmul(self.A_hat, self.D_neg_sqrt))
        support_2 = torch.matmul(support_1, torch.matmul(X, self.W))
        H = F.relu(support_2)

        return H

In [4]:
input_dim = 3
output_dim = 2
A = torch.tensor([[1., 0., 0.],
                  [0., 1., 1.],
                  [0., 1., 1.]])

gcn_layer = GCNLayer(input_dim, output_dim, A)

X = torch.tensor([[1., 2., 3.],
                  [4., 5., 6.],
                  [7., 8., 9.]])

output = gcn_layer(X)
print(output)

tensor([[ 6.5429,  4.0755],
        [12.3127,  9.1576],
        [14.1641, 10.6828]], grad_fn=<ReluBackward0>)
